### Define Read and format datasheets methods

In [ ]:
import pandas as pd

In [ ]:
# Define constants matching Fidelity download statement
FID_STOCKS_SECTION_HEADER = "Stocks"
FID_MUTUAL_FUNDS_SECTION_HEADER = "Mutual Funds"
FID_BEGINNING_VALUE_COLUMN = "Beginning Value"
FID_ENDING_VALUE_COLUMN = "Ending Value"
FID_COST_BASIS_COLUMN = "Cost Basis"

ABREVIATED_BEGINNING_VALUE_COLUMN = "Begin Val"
ABREVIATED_ENDING_VALUE_COLUMN = "End Val"
ABREVIATED_VALUE_ID = "Val ("

# Define a function to extract and format monthly stocks or mutual funds info from a file
def format_monthly_section_info(file_path, month_tag, section_header, preferred_order):
    """
    Extracts and formats a specific data section from a CSV file based on a header row,
    returning a cleaned and sorted pandas DataFrame.

    The function locates the start of the section using the specified header,
    trims the section based on blank rows or sentinel values (e.g., "Core Account", "Subtotal"),
    standardizes column names, filters key financial columns, and sorts based on a preferred symbol order.

    :param file_path: Path to the input CSV file.
    :type file_path: str
    :param section_header: String pattern identifying the beginning of the desired data section.
    :type section_header: str
    :param preferred_order: List of identifiers defining the sorting order for the Symbol/CUSIP column.
    :type preferred_order: list

    :return: Cleaned DataFrame containing selected financial columns in the preferred order.
    :rtype: pandas.DataFrame

    :Example:

    >>> df_cleaned = format_monthly_section_info(
    ...     "monthly_report.csv",
    ...     "Equities",
    ...     ["AAPL", "GOOG", "MSFT"]
    ... )
    """
    df = pd.read_csv(file_path)

    # Locate section start
    start_idx = df[df.iloc[:, 0].astype(str).str.contains(section_header, na=False)].index[0] + 1
    df_section = df.iloc[start_idx:].copy()

    # Stop at blank or next section
    stop_idx = df_section[
        df_section.iloc[:, 1].isna() |
        df_section.iloc[:, 1].astype(str).str.strip().eq("") |
        df_section.iloc[:, 1].astype(str).str.contains("Core Account|Subtotal", na=False)
    ].index
    if not stop_idx.empty:
        df_section = df_section.loc[:stop_idx[0] - 1]

    df_section = df_section.reset_index(drop=True)

    # Rename and clean columns
    df_section.columns = df.columns
    df_section.rename(columns={df.columns[0]: "Symbol/CUSIP", df.columns[1]: "Description"}, inplace=True)

    desired_columns = [
        "Symbol/CUSIP", "Description", "Quantity", "Price",
        FID_BEGINNING_VALUE_COLUMN, FID_ENDING_VALUE_COLUMN, FID_COST_BASIS_COLUMN
    ]
    df_section.columns = desired_columns + [f"drop_{i}" for i in range(len(df_section.columns) - len(desired_columns))]
    df_section = df_section[desired_columns]

    # Reorder columns
    column_order = [
        "Symbol/CUSIP", "Description", "Quantity", "Price",
        FID_COST_BASIS_COLUMN, FID_BEGINNING_VALUE_COLUMN, FID_ENDING_VALUE_COLUMN
    ]
    df_section = df_section[column_order]

    # Convert numeric columns to float where applicable
    numeric_columns = ["Quantity", "Price", FID_COST_BASIS_COLUMN, FID_BEGINNING_VALUE_COLUMN, FID_ENDING_VALUE_COLUMN]
    df_section[numeric_columns] = df_section[numeric_columns].apply(pd.to_numeric, errors='coerce')

    # Apply preferred order
    df_section["Symbol/CUSIP"] = pd.Categorical(df_section["Symbol/CUSIP"], categories=preferred_order, ordered=True)
    df_section = df_section.sort_values("Symbol/CUSIP").reset_index(drop=True)

    # Format and rename columns to include month tag
    df_section.rename(columns={
        FID_BEGINNING_VALUE_COLUMN: f"{ABREVIATED_BEGINNING_VALUE_COLUMN} ({month_tag})",
        FID_ENDING_VALUE_COLUMN: f"{ABREVIATED_ENDING_VALUE_COLUMN} ({month_tag})"
    }, inplace=True)

    return df_section

In [ ]:
# Define a function to extract monthly beginning and ending values for a given month
def extract_monthly_beginning_and_ending_values(df_stocks, month_tag):
    """
    Extract and rename beginning and ending value columns with a month tag.
    """
    return df_stocks[["Symbol/CUSIP", f"Beginning Value", f"Ending Value"]].rename(
        columns={
            "Beginning Value": f"Beginning Value ({month_tag})",
            "Ending Value": f"Ending Value ({month_tag})"
        }
    )

In [ ]:
# Define a function to extract a specific column from a formatted monthly df
def extract_and_rename_column(df_monthly, column_name, new_column_name):
    """
    Extract one column (e.g., 'Ending Value') from a formatted monthly df,
    rename it and return a DataFrame with Symbol/CUSIP + that column.
    """
    return df_monthly[["Symbol/CUSIP", column_name]].rename(columns={column_name: new_column_name})

In [ ]:
# Define a function to extract a specific column from a formatted monthly df
def extract_column(df_monthly, column_name):
    """
    Extract one column (e.g., 'Ending Value') from a formatted monthly df,
    and return a DataFrame with Symbol/CUSIP + that column.
    """
    return df_monthly[["Symbol/CUSIP", column_name]]

In [ ]:
# Define a function to append monthly values to the master_df
def append_monthly_values(master_df, monthly_values_df):
    """
    Join the monthly values to the master_df using 'Symbol/CUSIP' as key,
    and append the new columns at the end.
    """
    merged = pd.merge(master_df, monthly_values_df, on="Symbol/CUSIP", how="left")
    
    # Move newly added columns to the end
    fixed_cols = [col for col in master_df.columns]
    new_cols = [col for col in merged.columns if col not in fixed_cols and col != "Symbol/CUSIP"]
    final_cols = fixed_cols + new_cols
    return merged[final_cols]

### Execute read and formatting methods

In [ ]:
# Define the list of monthly files to process and their corresponding tags
monthly_reports_2025 = {
    "Jan": "Fid-IRA-2025-01-31.csv",
    "Feb": "Fid-IRA-2025-02-28.csv",
    "Mar": "Fid-IRA-2025-03-31.csv",
    "Apr": "Fid-IRA-2025-04-30.csv",
    "May": "Fid-IRA-2025-05-31.csv"
}

In [ ]:
monthly_reports_2024 = {
    "Jan": "Fid-IRA-2024-01-31.csv",
    "Feb": "Fid-IRA-2024-02-29.csv",
    "Mar": "Fid-IRA-2025-03-31.csv",
    "Apr": "Fid-IRA-2024-04-30.csv",
    "May": "Fid-IRA-2024-05-31.csv",
    "Jun": "Fid-IRA-2024-06-30.csv",
    "Jul": "Fid-IRA-2024-07-31.csv",
    "Aug": "Fid-IRA-2024-08-31.csv",
    "Sep": "Fid-IRA-2024-09-30.csv",
    "Oct": "Fid-IRA-2024-10-31.csv",
    "Nov": "Fid-IRA-2024-11-30.csv",
    "Dec": "Fid-IRA-2024-12-31.csv"
}

In [ ]:
monthly_reports = {
    "24Jan": "Fid-IRA-2024-01-31.csv",
    "24Feb": "Fid-IRA-2024-02-29.csv",
    "24Mar": "Fid-IRA-2025-03-31.csv",
    "24Apr": "Fid-IRA-2024-04-30.csv",
    "24May": "Fid-IRA-2024-05-31.csv",
    "24Jun": "Fid-IRA-2024-06-30.csv",
    "24Jul": "Fid-IRA-2024-07-31.csv",
    "24Aug": "Fid-IRA-2024-08-31.csv",
    "24Sep": "Fid-IRA-2024-09-30.csv",
    "24Oct": "Fid-IRA-2024-10-31.csv",
    "24Nov": "Fid-IRA-2024-11-30.csv",
    "24Dec": "Fid-IRA-2024-12-31.csv",
    "25Jan": "Fid-IRA-2025-01-31.csv",
    "25Feb": "Fid-IRA-2025-02-28.csv",
    "25Mar": "Fid-IRA-2025-03-31.csv",
    "25Apr": "Fid-IRA-2025-04-30.csv",
    "25May": "Fid-IRA-2025-05-31.csv"
}

In [ ]:
# Test integration of some functions without looping though months
def test_method_integration():
    # Step 1: Format full stock data for the current month
    df_jul = format_monthly_stock_info("Fid-IRA-2024-12-31.csv", "Dec")

    # Show full column list and the top few rows
    print("Columns:", df.columns.tolist())
    df_raw.head(15)
    
    # Step 2: Set up your master DataFrame based on July’s full data
    master_df = df_jul.copy()

    # Step 3: Extract just the monthly value columns for August
    df_jan = format_monthly_stock_info("Fid-IRA-2025-01-31.csv", "Jan")
    jan_beginining_ending_values_df = extract_monthly_beginning_and_ending_values(df_jan, "Jan")
    jan_ending_value = extract_monthly_column_values(df_jan, "Endign Value", "Jan")

    # Step 4: Append to master all
    master_all_df = append_monthly_values(master_df, jan_beginining_ending_values_df)
    display(master_all_df)
    # Step 4: Append to master ending
    master_ending_df = append_monthly_values(master_df, jan_beginining_ending_values_df)
    display(master_ending_df)
        
    # Step 2: Initialize the master dataframe
    master_df = None

    # Step 4: Loop through months and merge into master
    for month, filepath in monthly_reports.items():
        month_df = format_monthly_stock_info(filepath, month)

        if master_df is None:
            master_df = month_df
        else:
            price_colums_df = extract_monthly_column_values(month_df, "Ending Value", month)
            master_df = append_monthly_values(master_df, price_colums_df)
            
    # Optional: Fill missing values if any
    # master_df = master_df.fillna(0)

    # Preview final merged result
    pd.set_option('display.float_format', '${:,.2f}'.format)
    display(master_df)

In [ ]:
#  Read and Format ordered mutual funds price data for all the monthly_reports 

preferred_fund_order = [
    "FEMKX",  # FIDELITY EMERGING MARKETS
    "FNILX",  # FIDELITY ZERO LARGE CAP INDEX FUND
    "FSKAX",  # FIDELITY TOTAL MARKET INDEX FUND
    "FRESX",  # FIDELITY REAL ESTATE INVESTMENT
    "FSMDX",  # FIDELITY MID CAP INDEX FUND
    "FZILX",  # FIDELITY ZERO INTERNATIONAL INDEX
    "FIMVX",  # FIDELITY MID CAP VALUE INDEX FD
    "FAGIX",  # FIDELITY CAPITAL & INCOME
    "FXNAX"   # FIDELITY U.S. BOND INDEX FUND
]
# Step 2: Initialize the master dataframe
mutual_funds_df = None

# Step 4: Loop through months and merge into master
for month, filepath in monthly_reports.items():
    month_df = format_monthly_section_info(filepath, month, FID_MUTUAL_FUNDS_SECTION_HEADER, preferred_fund_order)

    if mutual_funds_df is None:
        mutual_funds_df = month_df
    else:
        price_colums_df = extract_column(month_df, f"{ABREVIATED_ENDING_VALUE_COLUMN} ({month})")
        mutual_funds_df = append_monthly_values(mutual_funds_df, price_colums_df)


# Optional: Fill missing values if any
# master_df = master_df.fillna(0)

# Preview final merged result
pd.set_option('display.float_format', '${:,.2f}'.format)
display(mutual_funds_df)

In [ ]:
# Read and Format ordered stocks price data for all the monthly_reports

preferred_stocks_order = [
    "ITOT",  # ISHARES CORE S&P TOTAL US STOCK MARKET ETF
    "IXUS",  # ISHARES TR CORE MSCI TOTAL
    "VWO",   # VANGUARD INTL EQUITY INDEX FDS FTSE EMR MKT ETF
    "VGK",   # VANGUARD INTL EQUITY INDEX FDS FTSE EUROPE ETF
    "VNQ",   # VANGUARD REAL ESTATE ETF
    "VTI",   # VANGUARD INDEX FDS VANGUARD TOTAL STK MKT ETF
    "VCLT",  # VANGUARD SCOTTSDALE FUNDS LONG-TERM CORP BOND IDX FUND
    "IAU",   # ISHARES GOLD TR ISHARES NEW ISIN #US4642852044 SEDOL #BKP74N6
    "GLDM",  # WORLD GOLD TR SPDR GLD MINIS
    "FCX",   # FREEPORT-MCMORAN INC COM USD0.10
    "KEY",   # KEYCORP COM
    "SSRM",  # SSR MINING INC COM NPV ISIN #CA7847301032 SEDOL #BF7MPL9
    "SNOW"   # SNOWFLAKE INC CL A
]

# Step 2: Initialize the master dataframe
stocks_df = None

# Step 4: Loop through months and merge into master
for month, filepath in monthly_reports.items():
    month_df = format_monthly_section_info(filepath, month, FID_STOCKS_SECTION_HEADER, preferred_stocks_order)

    if stocks_df is None:
        stocks_df = month_df
    else:
        price_colums_df = extract_column(month_df, f"{ABREVIATED_ENDING_VALUE_COLUMN} ({month})")
        stocks_df = append_monthly_values(stocks_df, price_colums_df)


# Optional: Fill missing values if any
# master_df = master_df.fillna(0)

# Preview final merged result
pd.set_option('display.float_format', '${:,.2f}'.format)
display(stocks_df)

In [ ]:
# Add Stock/Mutual Fund column as types to each dataframe
stocks_df["Type"] = "Stock"
mutual_funds_df["Type"] = "Fund"

# Combine dataframes into one 
df_combined = pd.concat([stocks_df, mutual_funds_df], ignore_index=True)

# Move 'Type' to the first column
cols = df_combined.columns.tolist()
cols.insert(0, cols.pop(cols.index("Type")))
df_combined = df_combined[cols]

display(df_combined)

### Define Plotting Routines

In [ ]:
# Plot
import matplotlib.pyplot as plt

In [ ]:
def plot_relative_performance(df_combined):
    # Select value columns (e.g., all beginning/ending value columns)
    value_cols = [col for col in df_combined.columns if ABREVIATED_VALUE_ID in col]

    # Create a new DataFrame with Symbol/CUSIP as index
    df_plot = df_combined.set_index("Symbol/CUSIP")[value_cols].copy()

    # Normalize to first column (starting value)
    df_normalized = df_plot.divide(df_plot.iloc[:, 0], axis=0) * 100

    df_normalized.T.plot(figsize=(12, 6), marker='o')
    plt.title("Relative Performance of Holdings")
    plt.xlabel("Month")
    plt.ylabel("Normalized Value (Base = 100)")
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()

In [ ]:
plot_relative_performance(df_combined)

In [ ]:
def plot_composition_over_time(df_combined):
    # Select value columns (e.g., all beginning/ending value columns)
    value_cols = [col for col in df_combined.columns if ABREVIATED_VALUE_ID in col]
    
    df_combined_by_type = df_combined.groupby("Type")[value_cols].sum()

    df_combined_by_type.T.plot.area(figsize=(12, 6))
    plt.title("Portfolio Composition Over Time")
    plt.xlabel("Month")
    plt.ylabel("Total Value")
    plt.legend(title="Asset Type")
    plt.tight_layout()
    plt.show()

In [ ]:
plot_composition_over_time(df_combined)

In [ ]:
def plot_composition_by_holding(df_combined):
    # Extract value columns (e.g., Beginning or Ending Value snapshots)
    value_cols = [col for col in df_combined.columns if ABREVIATED_VALUE_ID in col]

    # Pivot table: rows = Symbol/CUSIP, columns = Month
    df_area = df_combined.set_index("Symbol/CUSIP")[value_cols].apply(pd.to_numeric, errors='coerce')

    # Transpose for plotting
    df_area.T.plot.area(figsize=(14, 7), cmap='tab20')
    plt.title("Portfolio Composition by Holding Over Time")
    plt.xlabel("Month")
    plt.ylabel("Ending Value")
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', ncol=1)
    plt.tight_layout()
    plt.show()

In [ ]:
plot_composition_by_holding(df_combined)

In [ ]:
def plot_relative_performance_vs_cost_basis(df_combined):
    # Identify monthly Ending Value columns
    value_cols = [col for col in df_combined.columns if ABREVIATED_VALUE_ID in col]

    # Add Cost Basis as the first point in the timeline
    df_plot = df_combined.set_index("Symbol/CUSIP")[["Cost Basis"] + value_cols].copy()

    # Ensure all values are numeric
    df_plot = df_plot.apply(pd.to_numeric, errors="coerce")

    # Normalize each row by its Cost Basis
    df_normalized = df_plot.divide(df_plot["Cost Basis"], axis=0) * 100
    df_normalized = df_normalized.drop(columns="Cost Basis")  # Drop after using for normalization

    # Plot
    import matplotlib.pyplot as plt
    df_normalized.T.plot(figsize=(14, 6), marker='o')
    plt.title("Relative Performance vs Cost Basis (Base = 100)")
    plt.xlabel("Month")
    plt.ylabel("Normalized Value")
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()

In [ ]:
plot_relative_performance_vs_cost_basis(df_combined)

In [ ]:
def plot_composition_by_holding_including_cost_basis(df_combined):
    # Identify Begining or Ending Value columns
    value_cols = [col for col in df_combined.columns if ABREVIATED_VALUE_ID in col]

    # Include Cost Basis
    df_area = df_combined.set_index("Symbol/CUSIP")[["Cost Basis"] + value_cols].copy()

    # Ensure all numeric
    df_area = df_area.apply(pd.to_numeric, errors="coerce")

    df_area = df_area.T

    import matplotlib.pyplot as plt

    df_area.plot.area(figsize=(14, 7), cmap="tab20")
    plt.title("Portfolio Composition by Holding (Including Cost Basis)")
    plt.xlabel("Time")
    plt.ylabel("Dollar Value")
    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left", title="Holding")
    plt.tight_layout()
    plt.show()

In [ ]:
plot_composition_by_holding_including_cost_basis(df_combined)

In [ ]:
import seaborn as sns

def plot_month_over_month_performance(df_combined):
    df_pct_change = df_plot.pct_change(axis=1) * 100

    sns.heatmap(df_pct_change, annot=True, fmt=".1f", cmap="RdYlGn", center=0)
    plt.title("Month-over-Month Performance (%)")
    plt.xlabel("Month")
    plt.ylabel("Symbol")
    plt.tight_layout()
    plt.show()

In [ ]:
plot_relative_performance_vs_cost_basis(df_combined)

In [ ]:
plot_composition_by_holding_including_cost_basis(df_combined)

### Save combined datasheet to xcel file

In [ ]:
df_combined.to_excel("combined_holdings_24Jan-25May.xlsx", index=False)